In [1]:
%run init.ipynb

matchzoo version 0.0.1
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left=10,
    truncated_length_right=100,
    filter_low_freq=2
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 11021.37it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5931.83it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 3403984.01it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 11460.48it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 6061.15it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 949168.02it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 10314.56it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x10a6e70b8>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x130e43a90>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
fixed_padding_callback = mz.dataloader.callbacks.BasicPadding(
    fixed_length_left=10,
    fixed_length_right=100
)

In [7]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=fixed_padding_callback
)

testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=fixed_padding_callback
)

In [8]:
model = mz.models.ArcII()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['kernel_1d_count'] = 32
model.params['kernel_1d_size'] = 3
model.params['kernel_2d_count'] = [64, 64]
model.params['kernel_2d_size'] = [(3, 3), (3, 3)]
model.params['pool_2d_size'] = [(3, 3), (3, 3)]

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ArcII(
  (embedding): Embedding(16675, 100)
  (conv1d_left): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(100, 32, kernel_size=(3,), stride=(1,))
  )
  (conv1d_right): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(100, 32, kernel_size=(3,), stride=(1,))
  )
  (matching): Matching()
  (conv2d): Sequential(
    (0): Sequential(
      (0): ConstantPad2d(padding=(0, 2, 0, 2), value=0)
      (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
      (2): ReLU()
      (3): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): ConstantPad2d(padding=(0, 2, 0, 2), value=0)
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (2): ReLU()
      (3): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
    )
  )
  (dropout): Dropout(p=0.0, inplace=False)
  (out): Linear(in_features=704, out_features=1, bias=True)
)
Trainab

In [9]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-102 Loss-0.825]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.589 - normalized_discounted_cumulative_gain@5(0.0): 0.6432 - mean_average_precision(0.0): 0.6003



[Iter-204 Loss-0.487]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5976 - normalized_discounted_cumulative_gain@5(0.0): 0.6572 - mean_average_precision(0.0): 0.6063



[Iter-306 Loss-0.244]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5501 - normalized_discounted_cumulative_gain@5(0.0): 0.5998 - mean_average_precision(0.0): 0.558



[Iter-408 Loss-0.165]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5363 - normalized_discounted_cumulative_gain@5(0.0): 0.592 - mean_average_precision(0.0): 0.5457



[Iter-510 Loss-0.093]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.537 - normalized_discounted_cumulative_gain@5(0.0): 0.5912 - mean_average_precision(0.0): 0.5451



[Iter-612 Loss-0.058]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5375 - normalized_discounted_cumulative_gain@5(0.0): 0.5876 - mean_average_precision(0.0): 0.5436



[Iter-714 Loss-0.045]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5487 - normalized_discounted_cumulative_gain@5(0.0): 0.5925 - mean_average_precision(0.0): 0.5529



[Iter-816 Loss-0.036]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5342 - normalized_discounted_cumulative_gain@5(0.0): 0.5886 - mean_average_precision(0.0): 0.5402



[Iter-918 Loss-0.036]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.536 - normalized_discounted_cumulative_gain@5(0.0): 0.5925 - mean_average_precision(0.0): 0.5434



[Iter-1020 Loss-0.024]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5221 - normalized_discounted_cumulative_gain@5(0.0): 0.5786 - mean_average_precision(0.0): 0.537

Cost time: 500.527960062027s
